In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

/Users/luna/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_folder = "data/net_points/"
gender = "w" # Value to change if we want to analyze the women or the men
output_folder = f"results/net_points/{gender}/"

net_pts_women = pd.read_csv(data_folder + "charting-w-stats-NetPoints.csv")
net_pts_men = pd.read_csv(data_folder + "charting-m-stats-NetPoints.csv")

net_pts_w_or_m = net_pts_men if gender == "m" else net_pts_women

net_pts_w_or_m['year'] = net_pts_w_or_m['match_id'].str[:4]  
net_pts_w_or_m['tournament'] = net_pts_w_or_m['match_id'].str.split('-').str[2]  

net_pts_w_or_m_cleaned = net_pts_w_or_m[['year', 'tournament', 'player', 'row', 'net_pts', 'pts_won', 'net_winner', 'induced_forced', 'net_unforced', 'passed_at_net']]

net_pts_w_or_m_year = net_pts_w_or_m[['year', 'player']] ## add net_efficiency
net_pts_w_or_m_tournament = net_pts_w_or_m[['tournament', 'year']]
print(net_pts_w_or_m_cleaned)

       year tournament         player               row  net_pts  pts_won  \
0      2024      Wuhan  Magda Linette         NetPoints       11        8   
1      2024      Wuhan  Magda Linette          Approach        9        6   
2      2024      Wuhan  Magda Linette  NetPointsRallies       11        8   
3      2024      Wuhan  Magda Linette   ApproachRallies        9        6   
4      2024      Wuhan     Coco Gauff         NetPoints        9        6   
...     ...        ...            ...               ...      ...      ...   
23490  1979    US_Open   Tracy Austin   ApproachRallies        4        4   
23491  1979    US_Open    Chris Evert         NetPoints        8        3   
23492  1979    US_Open    Chris Evert          Approach        7        2   
23493  1979    US_Open    Chris Evert  NetPointsRallies        8        3   
23494  1979    US_Open    Chris Evert   ApproachRallies        7        2   

       net_winner  induced_forced  net_unforced  passed_at_net  
0         

In [3]:
# Separate the df for each categorie

net_pts_w_or_m_NetPoints = net_pts_w_or_m_cleaned[net_pts_w_or_m_cleaned['row'] == 'NetPoints'].drop(columns=['row'])
net_pts_w_or_m_Approach = net_pts_w_or_m_cleaned[net_pts_w_or_m_cleaned['row'] == 'Approach'].drop(columns=['row'])
net_pts_w_or_m_NetPointsRallies = net_pts_w_or_m_cleaned[net_pts_w_or_m_cleaned['row'] == 'NetPointsRallies'].drop(columns=['row'])
net_pts_w_or_m_ApproachRallies = net_pts_w_or_m_cleaned[net_pts_w_or_m_cleaned['row'] == 'ApproachRallies'].drop(columns=['row'])

players = net_pts_w_or_m['player'].unique()
tournaments = net_pts_w_or_m['tournament'].unique()
years = net_pts_w_or_m['year'].unique()

with open(output_folder + f'tournaments.json', 'w', encoding='utf-8') as f:
    json.dump(tournaments.tolist(), f, ensure_ascii=False, indent=2)
    
with open(output_folder + f'years.json', 'w', encoding='utf-8') as f:
    json.dump(years.tolist(), f, ensure_ascii=False, indent=2)


# 1. Net Efficiency

Who's the GOAT ? Who has the best net efficiency ? 

We conducted the analysis across different categories: Net Points, Approaches, Net Point Rallies, and Approach Rallies. Below is a brief explanation of each category:


   - Net Points: Points where a player finishes the rally at the net. This shows effectiveness in net play.
   - Approaches: Instances when a player intentionally moves toward the net to finish the point, usually following an approach shot.
   - Net Point Rallies: Rallies that involve at least one player coming to the net during the point. This captures the dynamics of net-based exchanges.
   - Approach Rallies: Rallies that begin with or involve an approach shot and result in net play. It highlights tactical use of approaches during rallies.

In [4]:
# compute net efficiency in multiple way 
# formula = (pts_won/net_pts) * 100
# 1. efficiency per player across all year and tournaments
# 2. efficiency per player for each tournament
# 3. efficiency per player for each year
# 4. efficiency per player for specific year - specific tournament 

def compute_net_efficiency(df, groupby):
    # threshold in order to have less biased data (depends on the number of matches played (only keep the player that played a lot of matches))

    df_efficiency = df.groupby(groupby).agg({
    'pts_won': 'sum',
    'net_pts': 'sum'
    }).reset_index()

    df_efficiency['net_efficiency'] = (df_efficiency['pts_won'] / df_efficiency['net_pts']) * 100
    threshold = df_efficiency['net_pts'].nlargest(10).iloc[-1] # Compute the threshold based on the 10 largest net points
    
    df_efficiency_filtered = df_efficiency[
        df_efficiency['net_pts'] >= threshold
    ].copy()

    df_efficiency_filtered = df_efficiency_filtered.sort_values(
        by='net_efficiency', ascending=False
    ).reset_index(drop=True)
    
    return df_efficiency_filtered, threshold

In [5]:
def overall(df, category): 
    
    df_overall, threshold = compute_net_efficiency(df, ['player'])
    file_name = output_folder + f"{category.lower()}/net_efficiency/overall_net_goats.csv"
    df_overall.to_csv(file_name, index=True)

In [6]:
# 2. per player for each tournament

tournament_list = []
def by_tournament(df, category): 
    df_by_tournament = {}

    for tournament in tournaments:
        df_tournament = df[df['tournament'] == tournament]
        if len(df_tournament) < 50: continue
        tournament_list.append(tournament)
        df_by_tournament[tournament], threshold = compute_net_efficiency(df_tournament, ['player'])

        file_name = output_folder + f"{category.lower()}/net_efficiency/net_goats_by_tournament/net_goats_{tournament}_tournament.csv"
        df_by_tournament[tournament].to_csv(file_name, index=True)
        
    with open(output_folder + f'{category}/tournaments.json', 'w', encoding='utf-8') as f:
        json.dump(tournaments.tolist(), f, ensure_ascii=False, indent=2)
    

In [7]:
# 3. per player for each year

def by_year(df, category):
    df_by_year = {}
    longueur = []

    for year in years:
        df_year = df[df['year'] == year]
        if len(df_year) < 50: continue
        df_by_year[year], threshold = compute_net_efficiency(df_year, ['player'])

        file_name = output_folder + f"{category.lower()}/net_efficiency/net_goats_by_year/net_goats_{year}.csv"
        df_by_year[year].to_csv(file_name, index=True)

In [8]:
# 4. Efficiency per player for each year and each tournament

def by_year_tournament(df, category):
    df_by_year_tournament = {}

    for year in years:
        df_year = df[df['year'] == year]

        for tournament in df_year['tournament'].unique():
            df_year_tournament = df_year[df_year['tournament'] == tournament]

            if len(df_year_tournament) < 50: continue

            df_efficiency, threshold = compute_net_efficiency(df_year_tournament, ['player'])

            df_by_year_tournament[(year, tournament)] = df_efficiency

            file_name = output_folder + f"{category.lower()}/net_efficiency/net_goats_by_year_tournament/net_goats_{year}_{tournament}.csv"
            df_efficiency.to_csv(file_name, index=True)


## 1.1. Net Points

In [9]:
# 1. efficiency per player across all year and tournaments

overall(net_pts_w_or_m_NetPoints,'Net Points')

In [10]:
# 2. net efficiency per player for each tournament

by_tournament(net_pts_w_or_m_NetPoints, 'Net Points')

In [11]:
# 3. Net efficiency per player for each year

by_year(net_pts_w_or_m_NetPoints, 'Net Points')

In [12]:
# 4. Net efficiency per player for each year and each tournament

by_year_tournament(net_pts_w_or_m_NetPoints, 'Net Points')

## 1.2. Approach

In [13]:
# 1. efficiency per player across all year and tournaments

overall(net_pts_w_or_m_Approach, 'Approach')

In [14]:
# 2. net efficiency per player for each tournament

by_tournament(net_pts_w_or_m_Approach, 'Approach')

In [15]:
# 3. Net efficiency per player for each year

by_year(net_pts_w_or_m_Approach, 'Approach')

In [16]:
# 4. Net efficiency per player for each year and each tournament

by_year_tournament(net_pts_w_or_m_Approach, 'Approach')

## 1.3. Net Points Rallies

In [17]:
# 1. efficiency per player across all year and tournaments

overall(net_pts_w_or_m_NetPointsRallies, 'Net Points Rallies')

In [18]:
# 2. net efficiency per player for each tournament

by_tournament(net_pts_w_or_m_NetPointsRallies, 'Net Points Rallies')

In [19]:
# 3. Net efficiency per player for each year

by_year(net_pts_w_or_m_NetPointsRallies, 'Net Points Rallies')

In [20]:
# 4. Net efficiency per player for each year and each tournament

by_year_tournament(net_pts_w_or_m_NetPointsRallies, 'Net Points Rallies')

## 1.4. Approach Rallies

In [21]:
# 1. efficiency per player across all year and tournaments

overall(net_pts_w_or_m_ApproachRallies, 'Approach Rallies')

In [22]:
# 2. net efficiency per player for each tournament

by_tournament(net_pts_w_or_m_ApproachRallies, 'Approach Rallies')

In [23]:
# 3. Net efficiency per player for each year

by_year(net_pts_w_or_m_ApproachRallies, 'Approach Rallies')

In [24]:
# 4. Net efficiency per player for each year and each tournament

by_year_tournament(net_pts_w_or_m_NetPointsRallies, 'Approach Rallies')

# 2. Passings Shots

Who's the GOAT ? Who's the best at handling passing shots ? 

The players with the lowest rate, are the best at handling the passing shots

Definition : a **passing shot** is a forceful shot, that travels to one side out of the reach of one's opponent. In tennis, this shot is generally a groundstroke and is used when one's opponent is running to the net or if they are at the net already.

In [25]:
# compute net efficiency in multiple way 
# formula = (pts_won/net_pts) * 100
# 1. efficiency per player across all year and tournaments
# 2. efficiency per player for each tournament
# 3. efficiency per player for each year
# 4. efficiency per player for specific year - specific tournament 

def compute_passing_shot(df, groupby):
    # threshold in order to have less biased data (depends on the number of matches played (only keep the player that played a lot of matches))

    df_passing_shot = df.groupby(groupby).agg({
    'passed_at_net': 'sum',
    'net_pts': 'sum'
    }).reset_index()

    df_passing_shot['passing_shot'] = (df_passing_shot['passed_at_net'] / df_passing_shot['net_pts']) * 100
    threshold = df_passing_shot['net_pts'].nlargest(10).iloc[-1] # Compute the threshold based on the 10 largest net points
    
    df_passing_shot_filtered = df_passing_shot[
        df_passing_shot['net_pts'] >= threshold
    ].copy()

    df_passing_shot_filtered = df_passing_shot_filtered.sort_values(
        by='passing_shot', ascending=False
    ).reset_index(drop=True)
    
    return df_passing_shot_filtered, threshold

In [26]:
def overall2(df, category, title): 
    
    df_overall, threshold = compute_passing_shot(df, ['player'])
    file_name = output_folder + f"{category.lower()}/passing_shots/overall_net_goats.csv"
    df_overall.to_csv(file_name, index=True)

In [27]:
# 2. per player for each tournament
tournament_list_passing_shots = []
def by_tournament2(df, category, title): 
    df_by_tournament = {}

    for tournament in tournaments:
        df_tournament = df[df['tournament'] == tournament]
        if len(df_tournament) < 50: continue
        tournament_list_passing_shots.append(tournament)
        df_by_tournament[tournament], threshold = compute_passing_shot(df_tournament, ['player'])


        file_name = output_folder + f"{category.lower()}/passing_shots/net_goats_by_tournament/net_goats_{tournament}_tournament.csv"
        df_by_tournament[tournament].to_csv(file_name, index=True)
        
    with open(output_folder + f'{category}/tournaments.json', 'w', encoding='utf-8') as f:
        json.dump(tournaments.tolist(), f, ensure_ascii=False, indent=2)

In [28]:
# 3. per player for each year

def by_year2(df, category, title):
    df_by_year = {}
    longueur = []

    for year in years:
        df_year = df[df['year'] == year]
        print(year, df_year)
        if len(df_year) < 50: continue
        df_by_year[year], threshold = compute_passing_shot(df_year, ['player'])

        file_name = output_folder + f"{category.lower()}/passing_shots/net_goats_by_year/net_goats_{year}.csv"
        df_by_year[year].to_csv(file_name, index=True)

In [29]:
# 4. Efficiency per player for each year and each tournament

def by_year_tournament2(df, category, title):
    df_by_year_tournament = {}

    for year in years:
        df_year = df[df['year'] == year]

        for tournament in df_year['tournament'].unique():
            df_year_tournament = df_year[df_year['tournament'] == tournament]

            if len(df_year_tournament) < 50: continue

            df_efficiency, threshold = compute_passing_shot(df_year_tournament, ['player'])

            df_by_year_tournament[(year, tournament)] = df_efficiency

            file_name = output_folder + f"{category.lower()}/passing_shots/net_goats_by_year_tournament/net_goats_{year}_{tournament}.csv"
            df_efficiency.to_csv(file_name, index=True)


## 2.1. Net Points

In [30]:
# 1. 

overall2(net_pts_w_or_m_NetPoints, 'Net Points', 'most resistant to passing shots')

In [31]:
# 2. 

by_tournament2(net_pts_w_or_m_NetPoints, 'Net Points', 'most resistant to passing shots')

In [32]:
#3. 

by_year2(net_pts_w_or_m_NetPoints, 'Net Points', 'most resistant to passing shots')

2024       year  tournament               player  net_pts  pts_won  net_winner  \
0     2024       Wuhan        Magda Linette       11        8           6   
4     2024       Wuhan           Coco Gauff        9        6           5   
8     2024       Wuhan        Marta Kostyuk        6        5           4   
12    2024       Wuhan           Coco Gauff       10        6           4   
16    2024       Wuhan       Cristina Bucsa        7        5           4   
...    ...         ...                  ...      ...      ...         ...   
2830  2024  United_Cup       Olga Danilovic       34       20          15   
2834  2024  United_Cup      Jasmine Paolini       11        8           5   
2838  2024  United_Cup      Caroline Garcia       17        9           5   
2842  2024  United_Cup          Iga Swiatek       18       12           7   
2846  2024  United_Cup  Sara Sorribes Tormo        2        2           1   

      induced_forced  net_unforced  passed_at_net  
0                 

2016        year             tournament                    player  net_pts  \
16401  2016           ITF_Valencia           Jasmine Paolini       25   
16405  2016           ITF_Valencia            Cristina Bucsa       20   
16409  2016  Fed_Cup_World_Group_F             Petra Kvitova        6   
16413  2016  Fed_Cup_World_Group_F           Caroline Garcia        8   
16417  2016  Fed_Cup_World_Group_F       Kristina Mladenovic       36   
...     ...                    ...                       ...      ...   
17084  2016               Auckland           Sloane Stephens       11   
17088  2016               Auckland          Jelena Ostapenko        6   
17092  2016               Auckland           Johanna Larsson        5   
17096  2016               Brisbane  Anastasia Pavlyuchenkova        8   
17100  2016               Brisbane          Timea Bacsinszky        4   

       pts_won  net_winner  induced_forced  net_unforced  passed_at_net  
16401       15          10               3  

21739          14               7             3              4  
2005        year         tournament             player  net_pts  pts_won  \
21743  2005  WTA_Championships    Amelie Mauresmo       32       21   
21747  2005  WTA_Championships        Mary Pierce       38       26   
21751  2005             Zurich  Lindsay Davenport       21       18   
21755  2005            US_Open        Mary Pierce       15        9   
21759  2005            US_Open      Kim Clijsters        4        4   
21763  2005            US_Open        Mary Pierce        8        2   
21767  2005            US_Open   Elena Dementieva       10        9   
21771  2005            US_Open    Serena Williams       10        5   
21775  2005            US_Open     Venus Williams       13       10   
21779  2005          Wimbledon     Venus Williams       23       18   
21783  2005          Wimbledon  Lindsay Davenport       14       11   
21787  2005          Wimbledon    Amelie Mauresmo       72       51   
21791  

1994        year       tournament                   player  net_pts  pts_won  \
22657  1994          US_Open  Arantxa Sanchez Vicario       23       14   
22661  1994          US_Open        Gabriela Sabatini       54       34   
22665  1994         Montreal              Steffi Graf       30       17   
22669  1994         Montreal  Arantxa Sanchez Vicario       39       22   
22673  1994           Mahwah              Steffi Graf       12        9   
22677  1994           Mahwah             Lisa Raymond       40       20   
22681  1994        Wimbledon        Conchita Martinez       16       11   
22685  1994        Wimbledon      Martina Navratilova      115       54   
22689  1994    Roland_Garros              Mary Pierce        7        6   
22693  1994    Roland_Garros  Arantxa Sanchez Vicario       20       12   
22697  1994    Amelia_Island              Mary Pierce       14        5   
22701  1994    Amelia_Island        Gabriela Sabatini       11        8   
22705  1994     Indi

1980        year tournament           player  net_pts  pts_won  net_winner  \
23479  1980    US_Open  Hana Mandlikova       86       46          22   
23483  1980    US_Open      Chris Evert        7        5           2   

       induced_forced  net_unforced  passed_at_net  
23479              13            14              9  
23483               3             0              1  
1979        year tournament        player  net_pts  pts_won  net_winner  \
23487  1979    US_Open  Tracy Austin        5        5           3   
23491  1979    US_Open   Chris Evert        8        3           2   

       induced_forced  net_unforced  passed_at_net  
23487               2             0              0  
23491               0             2              3  


In [33]:
#4. 

by_year_tournament2(net_pts_w_or_m_NetPoints, 'Net Points', 'most resistant to passing shots')

## 2.2. Approach

In [34]:
# 1. efficiency per player across all year and tournaments

overall2(net_pts_w_or_m_Approach, 'Approach', 'most resistant to passing shots')

In [35]:
# 2. net efficiency per player for each tournament

by_tournament2(net_pts_w_or_m_Approach, 'Approach', 'most resistant to passing shots')

In [36]:
# 3. Net efficiency per player for each year

by_year2(net_pts_w_or_m_Approach, 'Approach', 'most resistant to passing shots')

2024       year  tournament               player  net_pts  pts_won  net_winner  \
1     2024       Wuhan        Magda Linette        9        6           4   
5     2024       Wuhan           Coco Gauff        7        5           4   
9     2024       Wuhan        Marta Kostyuk        5        4           3   
13    2024       Wuhan           Coco Gauff        4        4           2   
17    2024       Wuhan       Cristina Bucsa        3        2           2   
...    ...         ...                  ...      ...      ...         ...   
2831  2024  United_Cup       Olga Danilovic       18        9           7   
2835  2024  United_Cup      Jasmine Paolini        1        1           1   
2839  2024  United_Cup      Caroline Garcia       15        8           4   
2843  2024  United_Cup          Iga Swiatek       15       10           6   
2847  2024  United_Cup  Sara Sorribes Tormo        2        2           1   

      induced_forced  net_unforced  passed_at_net  
1                 

2015        year tournament                player  net_pts  pts_won  net_winner  \
17105  2015   ITF_Waco     Viktorija Golubic        8        6           4   
17109  2015   ITF_Waco          Nicole Gibbs        5        1           0   
17113  2015     Zhuhai       Elina Svitolina        2        1           0   
17117  2015     Zhuhai     Karolina Pliskova        6        5           3   
17121  2015     Zhuhai       Elina Svitolina        9        4           2   
...     ...        ...                   ...      ...      ...         ...   
18855  2015   Auckland       Taylor Townsend        1        0           0   
18862  2015   Brisbane       Elina Svitolina        3        2           1   
18866  2015   Brisbane  Magdalena Rybarikova       12        4           2   
18874  2015   Auckland    Daniela Hantuchova        9        8           5   
18878  2015   Auckland           Sara Errani        1        1           0   

       induced_forced  net_unforced  passed_at_net  
17105

2008        year       tournament              player  net_pts  pts_won  \
21376  2008          US_Open     Serena Williams       16       13   
21380  2008          US_Open      Venus Williams       23       17   
21384  2008        New_Haven    Anna Chakvetadze        1        1   
21388  2008        New_Haven  Caroline Wozniacki        5        4   
21392  2008         Olympics       Dinara Safina        4        3   
21396  2008         Olympics    Elena Dementieva        9        6   
21400  2008         Olympics               Na Li        8        7   
21404  2008         Olympics       Dinara Safina        2        2   
21408  2008          US_Open     Serena Williams       23       20   
21412  2008          US_Open     Jelena Jankovic       14        8   
21416  2008        Wimbledon      Venus Williams       18       16   
21420  2008        Wimbledon     Serena Williams       13       10   
21424  2008    Roland_Garros       Dinara Safina        3        2   
21428  2008    

1999        year       tournament               player  net_pts  pts_won  \
22218  1999          Leipzig      Sandrine Testud       12        7   
22222  1999          Leipzig      Anna Kournikova       11       10   
22226  1999          US_Open      Serena Williams       13        9   
22230  1999          US_Open    Lindsay Davenport       19       12   
22234  1999        Wimbledon          Steffi Graf        8        7   
22238  1999        Wimbledon    Lindsay Davenport       20       13   
22242  1999        Wimbledon  Alexandra Stevenson        8        5   
22246  1999        Wimbledon    Lindsay Davenport        7        7   
22250  1999    Roland_Garros          Steffi Graf       17       12   
22254  1999    Roland_Garros       Martina Hingis       34       17   
22258  1999     Indian_Wells      Serena Williams       33       21   
22262  1999     Indian_Wells          Steffi Graf       13       10   
22266  1999          Hanover       Venus Williams       45       23   
2

1991        year                    tournament                   player  net_pts  \
22904  1991  Virginia_Slims_Championships      Martina Navratilova      146   
22908  1991  Virginia_Slims_Championships             Monica Seles        4   
22912  1991                      Brighton              Steffi Graf       34   
22916  1991                      Brighton            Zina Garrison       78   
22920  1991                       Leipzig             Jana Novotna       33   
22924  1991                       Leipzig              Steffi Graf        9   
22928  1991                       US_Open             Monica Seles        2   
22932  1991                       US_Open      Martina Navratilova       76   
22936  1991                     Wimbledon        Gabriela Sabatini       65   
22940  1991                     Wimbledon              Steffi Graf       38   
22944  1991                     Wimbledon        Jennifer Capriati       23   
22948  1991                     Wimbledon      

In [37]:
# 4. Net efficiency per player for each year and each tournament

by_year_tournament2(net_pts_w_or_m_Approach, 'Approach', 'most resistant to passing shots')

## 2.3. Net Points Rallies

In [38]:
# 1. efficiency per player across all year and tournaments

overall2(net_pts_w_or_m_NetPointsRallies, 'Net Points Rallies','most resistant to passing shots')

In [39]:
# 2. net efficiency per player for each tournament

by_tournament2(net_pts_w_or_m_NetPointsRallies, 'Net Points Rallies', 'most resistant to passing shots')

In [40]:
# 3. Net efficiency per player for each year

by_year2(net_pts_w_or_m_NetPointsRallies, 'Net Points Rallies','most resistant to passing shots')

2024       year  tournament               player  net_pts  pts_won  net_winner  \
2     2024       Wuhan        Magda Linette       11        8           6   
6     2024       Wuhan           Coco Gauff        9        6           5   
10    2024       Wuhan        Marta Kostyuk        6        5           4   
14    2024       Wuhan           Coco Gauff       10        6           4   
18    2024       Wuhan       Cristina Bucsa        6        5           4   
...    ...         ...                  ...      ...      ...         ...   
2832  2024  United_Cup       Olga Danilovic       34       20          15   
2836  2024  United_Cup      Jasmine Paolini       11        8           5   
2840  2024  United_Cup      Caroline Garcia       16        8           5   
2844  2024  United_Cup          Iga Swiatek       18       12           7   
2848  2024  United_Cup  Sara Sorribes Tormo        2        2           1   

      induced_forced  net_unforced  passed_at_net  
2                 

2017        year             tournament                 player  net_pts  pts_won  \
15577  2017  Fed_Cup_World_Group_F  Aliaksandra Sasnovich        2        2   
15581  2017  Fed_Cup_World_Group_F        Coco Vandeweghe        7        6   
15585  2017              Singapore     Caroline Wozniacki        5        2   
15589  2017              Singapore         Venus Williams       29       22   
15593  2017              Singapore        Caroline Garcia       17       11   
...     ...                    ...                    ...      ...      ...   
16385  2017                Sydney_          Johanna Konta        7        4   
16388  2017                 Sydney       Eugenie Bouchard        6        5   
16391  2017                 Sydney          Johanna Konta        7        4   
16395  2017               Brisbane        Elina Svitolina        9        5   
16399  2017               Brisbane      Karolina Pliskova       15       11   

       net_winner  induced_forced  net_unforce

2006        year             tournament               player  net_pts  pts_won  \
21637  2006      WTA_Championships        Nadia Petrova       32       21   
21641  2006      WTA_Championships       Martina Hingis       13        8   
21645  2006              Stuttgart      Tatiana Golovin       12        8   
21649  2006              Stuttgart   Michaella Krajicek        7        3   
21653  2006  Fed_Cup_World_Group_F        Justine Henin       22       16   
21657  2006  Fed_Cup_World_Group_F  Francesca Schiavone       21       10   
21661  2006                US_Open      Amelie Mauresmo       26       12   
21665  2006                US_Open      Maria Sharapova        7        6   
21669  2006                US_Open        Justine Henin       12       12   
21673  2006                US_Open          Shahar Peer        6        3   
21676  2006              New_Haven      Amelie Mauresmo        6        3   
21678  2006              New_Haven    Lindsay Davenport        6       

1982        year       tournament               player  net_pts  pts_won  \
23433  1982  Australian_Open  Martina Navratilova       33       18   
23437  1982  Australian_Open          Chris Evert       37       24   
23441  1982          US_Open      Hana Mandlikova       30       19   
23445  1982          US_Open          Chris Evert       13       11   

       net_winner  induced_forced  net_unforced  passed_at_net  
23433          12               5             5              5  
23437          10               5             4              5  
23441           9               7             0              4  
23445           6               4             0              1  
1981        year       tournament               player  net_pts  pts_won  \
23449  1981  Australian_Open  Martina Navratilova       71       38   
23453  1981  Australian_Open          Chris Evert       38       20   
23457  1981            Perth            Eva Pfaff        4        2   
23461  1981            Pe

In [41]:
# 4. Net efficiency per player for each year and each tournament

by_year_tournament2(net_pts_w_or_m_NetPointsRallies, 'Net Points Rallies','most resistant to passing shots')

## 2.4. Approach Rallies

In [42]:
# 1. efficiency per player across all year and tournaments

overall2(net_pts_w_or_m_ApproachRallies, 'Approach Rallies','most resistant to passing shots')

In [43]:
# 2. net efficiency per player for each tournament

by_tournament2(net_pts_w_or_m_ApproachRallies, 'Approach Rallies', 'most resistant to passing shots')

In [44]:
# 3. Net efficiency per player for each year

by_year2(net_pts_w_or_m_ApproachRallies, 'Approach Rallies', 'most resistant to passing shots')

2024       year  tournament               player  net_pts  pts_won  net_winner  \
3     2024       Wuhan        Magda Linette        9        6           4   
7     2024       Wuhan           Coco Gauff        7        5           4   
11    2024       Wuhan        Marta Kostyuk        5        4           3   
15    2024       Wuhan           Coco Gauff        4        4           2   
19    2024       Wuhan       Cristina Bucsa        2        2           2   
...    ...         ...                  ...      ...      ...         ...   
2833  2024  United_Cup       Olga Danilovic       18        9           7   
2837  2024  United_Cup      Jasmine Paolini        1        1           1   
2841  2024  United_Cup      Caroline Garcia       14        7           4   
2845  2024  United_Cup          Iga Swiatek       15       10           6   
2849  2024  United_Cup  Sara Sorribes Tormo        2        2           1   

      induced_forced  net_unforced  passed_at_net  
3                 

2014        year tournament                player  net_pts  pts_won  net_winner  \
18888  2014      Sofia       Flavia Pennetta       12        9           6   
18892  2014      Sofia  Carla Suarez Navarro        4        2           2   
18896  2014      Sofia    Tsvetana Pironkova        2        1           0   
18900  2014      Sofia    Dominika Cibulkova        8        6           4   
18904  2014      Sofia         Kate Makarova        2        2           2   
...     ...        ...                   ...      ...      ...         ...   
19834  2014   Auckland          Lauren Davis        2        1           1   
19838  2014   Brisbane      Angelique Kerber       10        6           5   
19842  2014   Brisbane       Jelena Jankovic       12        8           5   
19846  2014   Auckland      Garbine Muguruza        1        0           0   
19850  2014   Auckland        Venus Williams        4        3           2   

       induced_forced  net_unforced  passed_at_net  
18888

1987        year       tournament               player  net_pts  pts_won  \
23234  1987          US_Open          Steffi Graf        6        4   
23238  1987          US_Open  Martina Navratilova       24       14   
23242  1987        Wimbledon          Chris Evert       33       19   
23246  1987        Wimbledon  Martina Navratilova       46       24   
23250  1987        Wimbledon  Martina Navratilova       12        5   
23254  1987        Wimbledon          Steffi Graf       12        7   
23258  1987    Roland_Garros          Steffi Graf       18        8   
23262  1987    Roland_Garros  Martina Navratilova       45       23   
23266  1987     Key_Biscayne          Chris Evert       14        7   
23270  1987     Key_Biscayne          Steffi Graf        2        2   
23274  1987  Australian_Open      Hana Mandlikova        8        4   
23278  1987  Australian_Open  Martina Navratilova       15       10   

       net_winner  induced_forced  net_unforced  passed_at_net  
23234 

In [45]:
# 4. Net efficiency per player for each year and each tournament

by_year_tournament2(net_pts_w_or_m_ApproachRallies, 'Approach Rallies', 'most resistant to passing shots')